# Particle Sand

Using penalty-based collisions to simulate particle sand. 
When a collision is detected, the model adds a virtual spring of rest length 0 between the colliding objects

In [ ]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10
screen_size = 600
bowl_r = 400
c = ti.Vector([0.5, 0.75]) # center of the bowl

ks = 1e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)


@ti.kernel
def substep():
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,-2] # reset force, add in gravity
        # boundary check with the bowl
        phi = bowl_r - ((x[i] - c).norm() * screen_size) - r[i]
        neg_grad = (x[i] - c).normalized()
        if phi < 0: # outside of the bowl
            f[i] += ks * (phi) * neg_grad
            f[i] += kd * (ti.Vector([0, 0]) - v[i]).dot(neg_grad) * neg_grad
            
        for j in range(i+1, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/screen_size - r[j]/screen_size
            normal = (x[j] - x[i]).normalized()
            if phi < 0:
                force_spring = ks * (phi) * normal
                force_damp = kd * (v[j] - v[i]).dot(normal) * normal
                f[i] += force_spring
                f[j] -= force_spring
                f[i] += force_damp
                f[j] -= force_damp
                
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / m[i]
        # update x
        x[i] = x[i] + dt * v[i]

        
    ### WEIRD BEHAVIOR
    # The sand seems to gain velocity when hitting the right wall
    # collisions aren't working - if one has no x component, it'll only sometimes gain x 
    #   velocity when it collides with another particle
    ###
        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Particle Sand', (screen_size, screen_size))
    
    new_particle(0.3, 0.5)
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


### Moving the particles with the mouse

UNFINISHED

In [ ]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10
screen_size = 400

ks = 2e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)


@ti.kernel
def substep():
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,-2] # reset force, add in gravity
        # boundary check
        phi = x[i][1] - r[i]/screen_size
        if phi < 0: # lower boundary
            f[i] += ks * (phi) * ti.Vector([0,-1])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
        phi = x[i][0] - r[i]/screen_size
        if phi < 0: # left boundary
            f[i] += ks * (phi) * ti.Vector([-1,0])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([-1,0])) * ti.Vector([-1,0])
        phi = x[i][0] + r[i]/screen_size
        if phi > 1:
            f[i] += ks * (phi) * ti.Vector([-1,0])
            f[i] -= kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([-1,0])) * ti.Vector([-1,0])
            ###
            # How do I make it so the resting state isn't slightly submerged
            ###

        for j in range(i+1, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/screen_size - r[j]/screen_size
            normal = (x[j] - x[i]).normalized()
            if phi < 0:
                force_spring = ks * (phi) * normal
                force_damp = kd * (v[j] - v[i]).dot(normal) * normal
                f[i] += force_spring
                f[j] -= force_spring
                f[i] += force_damp
                f[j] -= force_damp
                
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / m[i]
        # update x
        x[i] = x[i] + dt * v[i]
        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

class MouseDataGen:
    def __init__(self):
        self.prev_mouse = None

    def __call__(self, gui):
        # [0:2]: normalized delta direction
        # [2:4]: current mouse xy
        # [4:7]: color
        mouse_data = np.zeros(8, dtype=np.float32)
        if gui.is_pressed(ti.GUI.LMB):
            mxy = np.array(gui.get_cursor_pos(), dtype=np.float32) * res
            if self.prev_mouse is None:
                self.prev_mouse = mxy
            else:
                mdir = mxy - self.prev_mouse
                mdir = mdir / (np.linalg.norm(mdir) + 1e-5)
                mouse_data[0], mouse_data[1] = mdir[0], mdir[1]
                mouse_data[2], mouse_data[3] = mxy[0], mxy[1]
                self.prev_mouse = mxy
        else:
            self.prev_mouse = None
        return mouse_data
        
        
def main():
    gui = ti.GUI('Particle Sand', (screen_size, screen_size))
    
    new_particle(0.3, 0.5)
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()